In [1]:
import pandas as pd
import os
import requests
import admin
import geocoder as gc
import time

In [2]:
df1 = admin.read_csv_files(r"C:\Users\THW_9\Desktop\LSE School\Dis\REALIS DATA\2024")

In [3]:
df1 ['Street_Address'] = df1['Address'].str.split(' #').str[0]
df_street_address = df1.groupby('Street_Address').size().reset_index(name='count')

In [4]:
df_dist = pd.read_csv(r"C:\Users\THW_9\Desktop\LSE School\Dis\GIS\Historic\street_addresses_1846RD.csv")

In [5]:
# replace NaN with zeros in 'Polygon' column
df_dist['Polygon'] = df_dist['Polygon'].fillna(0)

df_dist['signed_distance'] = df_dist['Distance'] * (df_dist['Polygon'] - 1)  # Create a new column for signed distance

#if polygon is 1, make distance negative
df_dist.loc[df_dist['Polygon'] == 1, 'signed_distance'] = df_dist['Distance']

In [6]:
#clean up for duplicate 'InputID'

df_dist = df_dist.drop_duplicates(subset=['InputID'])
df_dist

,InputID,TargetID,Distance,Polygon,signed_distance
0,1 AKYAB ROAD,1,2001.530092,0.0,-2001.530092
1,1 ALEXANDRA VIEW,1,3015.441745,0.0,-3015.441745
2,1 AMBER GARDENS,1,3616.726441,0.0,-3616.726441
3,1 AMBER ROAD,1,3696.611665,0.0,-3696.611665
4,1 ANG MO KIO STREET 66,1,9446.531940,0.0,-9446.531940
...,...,...,...,...,...
6351,9D JALAN BERJAYA,1,5431.025598,0.0,-5431.025598
6352,9D RECREATION LANE,1,4998.050263,0.0,-4998.050263
6353,9E HILLCREST ROAD,1,5287.597890,0.0,-5287.597890
6354,9G YUAN CHING ROAD,1,13780.568969,0.0,-13780.568969


In [8]:
# merge df1 'Address_Rd' with df_dist 'InputID'
df2 = df_street_address.merge(df_dist, left_on='Street_Address', right_on='InputID', how='left')

In [9]:
df2

,Street_Address,count,InputID,TargetID,Distance,Polygon,signed_distance
0,1 AKYAB ROAD,1,1 AKYAB ROAD,1.0,2001.530092,0.0,-2001.530092
1,1 ALEXANDRA VIEW,9,1 ALEXANDRA VIEW,1.0,3015.441745,0.0,-3015.441745
2,1 AMBER GARDENS,8,1 AMBER GARDENS,1.0,3616.726441,0.0,-3616.726441
3,1 AMBER ROAD,2,1 AMBER ROAD,1.0,3696.611665,0.0,-3696.611665
4,1 ANG MO KIO STREET 66,3,1 ANG MO KIO STREET 66,1.0,9446.531940,0.0,-9446.531940
...,...,...,...,...,...,...,...
6320,9D JALAN BERJAYA,1,9D JALAN BERJAYA,1.0,5431.025598,0.0,-5431.025598
6321,9D RECREATION LANE,1,9D RECREATION LANE,1.0,4998.050263,0.0,-4998.050263
6322,9E HILLCREST ROAD,1,9E HILLCREST ROAD,1.0,5287.597890,0.0,-5287.597890
6323,9G YUAN CHING ROAD,2,9G YUAN CHING ROAD,1.0,13780.568969,0.0,-13780.568969


In [10]:
df_MRTdist = pd.read_csv(r"C:\Users\THW_9\Desktop\LSE School\Dis\GIS\Historic\street_addresses_1846RD_reproject_k5MRTdist.csv")

In [11]:
df_MRTdist

,InputID,TargetID,Distance
0,1 AKYAB ROAD,NOVENA MRT STATION,497.245324
1,1 AKYAB ROAD,TOA PAYOH MRT STATION,1100.477899
2,1 AKYAB ROAD,HALIFAX SUB STATION CABLE TROUGH,1323.895794
3,1 AKYAB ROAD,NEWTON MRT STATION,1475.411563
4,1 AKYAB ROAD,FARRER PARK MRT STATION,1354.034075
...,...,...,...
63555,C-1 HOLLAND PARK,DHOBY GHAUT MRT STATION,266.878095
63556,C-1 HOLLAND PARK,BENCOOLEN MRT STATION,749.909965
63557,C-1 HOLLAND PARK,DHOBY GHAUT MRT STATION,311.482130
63558,C-1 HOLLAND PARK,DHOBY GHAUT MRT STATION,209.671512


In [12]:
# groupby 'InputID' and min 'Distance'

df_MRTdist = df_MRTdist.groupby('InputID', as_index=False).agg({'Distance': 'min'})

In [14]:
#rename "Distance" to "MRT_Distance"
df_MRTdist = df_MRTdist.rename(columns={'Distance': 'MRT_Distance'})
df_MRTdist


,InputID,MRT_Distance
0,1 AKYAB ROAD,222.262798
1,1 ALEXANDRA VIEW,148.386568
2,1 AMBER GARDENS,236.388779
3,1 AMBER ROAD,131.625919
4,1 ANG MO KIO STREET 66,159.594288
...,...,...
6313,9D JALAN BERJAYA,222.262798
6314,9D RECREATION LANE,198.981838
6315,9E HILLCREST ROAD,209.671512
6316,9G YUAN CHING ROAD,209.671512


In [15]:
#merge df2 'InputID' with df_MRTdist 'InputID'
df2 = df2.merge(df_MRTdist, left_on='InputID', right_on='InputID', how='left')

In [17]:
#merge df1 'street_address' with df2 'Street_Address'
df1 = df1.merge(df2, left_on='Street_Address', right_on='Street_Address', how='left')

In [19]:
#drop Null values in 'MRT_Distance' and 'signed_distance'
df1 = df1.dropna(subset=['MRT_Distance', 'signed_distance'])
df1

,Project Name,Transacted Price ($),Area (SQFT),Unit Price ($ PSF),Sale Date,Address,Type of Sale,Type of Area,Area (SQM),Unit Price ($ PSM),...,Planning Region,Planning Area,Street_Address,count,InputID,TargetID,Distance,Polygon,signed_distance,MRT_Distance
0,THE LANDMARK,"2,726,888","1,076.4","2,533",01 Jan 2024,173 CHIN SWEE ROAD #22-11,New Sale,Strata,100,"27,269",...,Central Region,Outram,173 CHIN SWEE ROAD,75,173 CHIN SWEE ROAD,1.0,526.930012,0.0,-526.930012,56.788496
1,POLLEN COLLECTION,"3,850,000","1,808.35","2,129",01 Jan 2024,34 POLLEN PLACE,New Sale,Land,168,"22,917",...,North East Region,Serangoon,34 POLLEN PLACE,1,34 POLLEN PLACE,1.0,8197.955130,0.0,-8197.955130,159.594288
2,SKY EDEN@BEDOK,"2,346,000","1,087.16","2,158",01 Jan 2024,1 BEDOK CENTRAL #09-10,New Sale,Strata,101,"23,228",...,East Region,Bedok,1 BEDOK CENTRAL,13,1 BEDOK CENTRAL,1.0,7709.723490,0.0,-7709.723490,265.149104
3,TERRA HILL,"2,190,000",807.3,"2,713",01 Jan 2024,18A YEW SIANG ROAD #03-12,New Sale,Strata,75,"29,200",...,Central Region,Queenstown,18A YEW SIANG ROAD,4,18A YEW SIANG ROAD,1.0,5654.833490,0.0,-5654.833490,56.788496
4,PINETREE HILL,"1,954,000",796.54,"2,453",01 Jan 2024,36 PINE GROVE #05-18,New Sale,Strata,74,"26,405",...,Central Region,Bukit Timah,36 PINE GROVE,54,36 PINE GROVE,1.0,7770.291003,0.0,-7770.291003,209.671512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25089,SAVOY PARK,"4,050,000","1,979.5","2,046",31 Dec 2024,16 ENG KONG DRIVE,Resale,Land,183.9,"22,023",...,Central Region,Bukit Timah,16 ENG KONG DRIVE,1,16 ENG KONG DRIVE,1.0,9148.170194,0.0,-9148.170194,209.671512
25090,THE HILL @ONE-NORTH,"2,595,000","1,033.34","2,511",31 Dec 2024,17 SLIM BARRACKS RISE #09-15,New Sale,Strata,96,"27,031",...,Central Region,Queenstown,17 SLIM BARRACKS RISE,29,17 SLIM BARRACKS RISE,1.0,5949.512812,0.0,-5949.512812,209.671512
25091,HILLTOPS,"5,320,000","1,593.07","3,339",31 Dec 2024,101 CAIRNHILL CIRCLE #06-03,Resale,Strata,148,"35,946",...,Central Region,Newton,101 CAIRNHILL CIRCLE,11,101 CAIRNHILL CIRCLE,1.0,984.294040,0.0,-984.294040,209.671512
25092,SERANGOON GARDEN ESTATE,"7,880,000","4,199.04","1,877",31 Dec 2024,5 MEDWAY DRIVE,Resale,Land,390.1,"20,200",...,North East Region,Serangoon,5 MEDWAY DRIVE,1,5 MEDWAY DRIVE,1.0,6090.563133,0.0,-6090.563133,198.981838
